In [13]:
import time

import numpy as np
import plotly.express as px
from scipy.sparse import random as sparse_random

In [14]:
def rsvd(A, rank, power_iterations=3):
    """
    Perform Randomized Singular Value Decomposition (RSVD).

    Parameters:
        A (np.ndarray): Input matrix.
        rank (int): Target rank for the approximation.
        power_iterations (int): Number of power iterations to enhance accuracy.

    Returns:
        u (np.ndarray): Left singular vectors.
        s (np.ndarray): Singular values.
        v (np.ndarray): Right singular vectors (transposed).
    """
    # Step 1: Generate a random matrix Omega
    n_rows, n_cols = A.shape
    Omega = np.random.randn(n_cols, rank)

    # Step 2: Perform power iteration
    Y = A @ Omega
    for _ in range(power_iterations):
        Y = A @ (A.T @ Y)

    # Step 3: Compute orthogonal matrix Q
    Q, _ = np.linalg.qr(Y)

    # Step 4: Project A onto the low-dimensional subspace
    B = Q.T @ A

    # Step 5: Compute SVD on the smaller matrix B
    u_tilde, s, v = np.linalg.svd(B, full_matrices=False)

    # Step 6: Recover the left singular vectors of A
    u = Q @ u_tilde

    return u, s, v

In [15]:
def generate_sparse_matrices(start_size, end_size, step, sparsity, random_state=42):
    """
    Generate a list of sparse matrices similar to the logic in the C++ code.

    Parameters:
        start_size (int): The starting matrix size.
        end_size (int): The ending matrix size.
        step (int): Step size for incrementing the matrix size.
        sparsity (float): Probability of nonzero elements.
        random_state (int): Random seed for reproducibility.

    Returns:
        matrices (list of scipy.sparse.csr_matrix): A list of generated sparse matrices.
    """
    matrices = []
    rs = np.random.RandomState(random_state)
    for size in range(start_size, end_size + 1, step):
        # Use scipy.sparse.random to generate a random sparse matrix
        # 'density' corresponds to sparsity (fraction of nonzero entries)
        sparse_mat = sparse_random(
            size, size, density=sparsity, random_state=rs, data_rvs=rs.rand
        )
        matrices.append(sparse_mat.tocsr())
    return matrices


# Parameters to match the C++ code
startSize = 100
endSize = 1000
stepSize = 100
sparsity = 0.1

In [16]:
matrices = generate_sparse_matrices(startSize, endSize, stepSize, sparsity)

# Test RSVD performance on these matrices
rank = 10  # example: target rank for the approximation
times = []
sizes = []

In [17]:
for mat in matrices:
    # Convert sparse matrix to dense array for the current RSVD implementation
    A = mat.toarray()
    start = time.time()
    u, s, v = rsvd(A, rank=rank)
    end = time.time()
    elapsed = (end - start) * 1e9  # Convert to nanoseconds for comparison
    times.append(elapsed)
    sizes.append(A.shape[0])

# Plot the results using Plotly
fig = px.line(
    x=sizes,
    y=times,
    title="RSVD Average Execution Time vs Matrix Size (Python)",
    labels={"x": "Matrix Size", "y": "Execution Time (ns)"},
)
fig.show()

In [18]:
%%time
# Perform RSVD
_, _, _ = rsvd(A, rank, power_iterations=2)

CPU times: user 3.61 ms, sys: 2.09 ms, total: 5.7 ms
Wall time: 4.13 ms


In [19]:
%%time
np.linalg.svd(A, full_matrices=False)

CPU times: user 284 ms, sys: 61.9 ms, total: 346 ms
Wall time: 257 ms


SVDResult(U=array([[-0.02378277, -0.00291523, -0.01609171, ...,  0.02588873,
         0.05323179, -0.00119431],
       [-0.02790633, -0.04423366, -0.04435035, ...,  0.02669706,
        -0.01837708,  0.03972077],
       [-0.02967725,  0.01591466,  0.01057777, ..., -0.00973599,
        -0.02512284,  0.03942442],
       ...,
       [-0.02729643,  0.0235439 ,  0.00022777, ...,  0.01295292,
        -0.04682329,  0.05172808],
       [-0.02708616, -0.01581963, -0.01447751, ..., -0.05759904,
         0.04394445,  0.04437689],
       [-0.03491446,  0.01993703, -0.03359398, ...,  0.00755171,
         0.01473208,  0.00395821]], shape=(1000, 1000)), S=array([5.05918345e+01, 1.10988487e+01, 1.10264005e+01, 1.09467073e+01,
       1.09041175e+01, 1.08920980e+01, 1.08789945e+01, 1.08246400e+01,
       1.07895039e+01, 1.07742294e+01, 1.07146821e+01, 1.06989134e+01,
       1.06568968e+01, 1.06124493e+01, 1.06044243e+01, 1.05768405e+01,
       1.05627917e+01, 1.05337973e+01, 1.05096349e+01, 1.05056109e+0

In [20]:
import numpy as np

A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], dtype=float)

rank = 2

U, s, Vt = np.linalg.svd(A, full_matrices=False)

s_k = s[:rank]
U_k = U[:, :rank]
V_k = Vt[:rank, :].T

print("Singular values:\n", s_k, "\n")
print("U:\n", U_k, "\n")
print("V:\n", V_k, "\n")

Singular values:
 [25.46240744  1.29066168] 

U:
 [[-0.14087668 -0.82471435]
 [-0.34394629 -0.42626394]
 [-0.54701591 -0.02781353]
 [-0.75008553  0.37063688]] 

V:
 [[-0.50453315  0.76077568]
 [-0.5745157   0.05714052]
 [-0.64449826 -0.64649464]] 

